In [2]:
# deploy.py
import uuid
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    Model,
    Environment,
    CodeConfiguration,
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
)

# 🔐 Azure Configuration
subscription_id = "1866914e-e6c4-4ac4-ac02-edbc34859691"
resource_group = "newml"
workspace_name = "mlspace1"

# Connect to workspace
ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id,
    resource_group,
    workspace_name
)



In [4]:
# registering both models by combining
model = ml_client.models.create_or_update(
    Model(path="./model_assets", name="CombinedModel")
)


Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy '/mnt/batch/tasks/shared/LS_root/mounts/clusters/shubhamss21091/code/Users/shubhamss2109/model_assets' 'https://mlspace18808489114.blob.core.windows.net/azureml-blobstore-7ef0e3f6-a4a5-4403-b8b6-bf98a71e3316/LocalUpload/f434bd7f1eb9c9e932ff2994cdb4a8e107b8bb66d8561e2ec46d5daa0f96e77e/model_assets' 

See https://learn.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.


In [5]:
# 2️⃣ Create the environment
env = Environment(
    name="my-env4",
    description="Environment for sklearn model",
    conda_file="environment.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest"
)
ml_client.environments.create_or_update(env)


Environment({'arm_type': 'environment_version', 'latest_version': None, 'image': 'mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest', 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'my-env4', 'description': 'Environment for sklearn model', 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': False, 'id': '/subscriptions/1866914e-e6c4-4ac4-ac02-edbc34859691/resourceGroups/newml/providers/Microsoft.MachineLearningServices/workspaces/mlspace1/environments/my-env4/versions/1', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/shubhamss21091/code/Users/shubhamss2109', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7dbbb7903310>, 'serialize': <msrest.serialization.Serializer object at 0x7dbbb7901210>, 'version': '1', 'conda_file': {'channels': ['defaults'], 'dependencies': ['python= 3.10.11', 'pip', {'pip': ['scikit-l

In [6]:
# 3️⃣ Create the endpoint
endpoint = ManagedOnlineEndpoint(
    name="newendpoint1",
    description="Scikit-learn new model endpoint",
    auth_mode="key"
)
ml_client.online_endpoints.begin_create_or_update(endpoint).result()


/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/mlflow/__init__.py:41: UserWarning: Versions of mlflow (3.1.1) and mlflow-skinny (2.22.1) are different. This may lead to unexpected behavior. Please install the same version of both packages.
  mlflow.mismatch._check_version_mismatch()


ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://newendpoint1.westus3.inference.ml.azure.com/score', 'openapi_uri': 'https://newendpoint1.westus3.inference.ml.azure.com/swagger.json', 'name': 'newendpoint1', 'description': 'Scikit-learn new model endpoint', 'tags': {}, 'properties': {'createdBy': 'SHUBHAM SHARMA', 'createdAt': '2025-09-05T15:35:29.916696+0000', 'lastModifiedAt': '2025-09-05T15:35:29.916696+0000', 'azureml.onlineendpointid': '/subscriptions/1866914e-e6c4-4ac4-ac02-edbc34859691/resourcegroups/newml/providers/microsoft.machinelearningservices/workspaces/mlspace1/onlineendpoints/newendpoint1', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/1866914e-e6c4-4ac4-ac02-edbc34859691/providers/Microsoft.MachineLearningServices/locations/westus3/mfeOperationsStatus/oeidp:7ef0e3f6-a4a5-4403-b8b6-bf98a71e3316:8d784fb1-0e9a-4c88-aca4-3370316783ac?api-version=2022-02-01-preview'}, 'print_as_yaml

In [7]:
from azure.ai.ml.entities import ManagedOnlineDeployment, CodeConfiguration, Model

deployment = ManagedOnlineDeployment(
    name="green1",
    endpoint_name="newendpoint1",
    model=model.id,   
    environment=env,
    code_configuration=CodeConfiguration(
        code=".",  # folder containing score.py
        scoring_script="score.py",
    ),
    instance_type="Standard_DS2_v2",
    instance_count=1,
)

ml_client.online_deployments.begin_create_or_update(deployment).result()


Instance type Standard_DS2_v2 may be too small for compute resources. Minimum recommended compute SKU is Standard_DS3_v2 for general purpose endpoints. Learn more about SKUs here: https://learn.microsoft.com/azure/machine-learning/referencemanaged-online-endpoints-vm-sku-list
Check: endpoint newendpoint1 exists
